In [1]:
import pandas as pd
import csv
from tqdm import tqdm
import json
from glob import glob
from uuid import uuid5, NAMESPACE_URL

In [15]:
name_id_mapper = {}
id_mapper = {}
for filename in tqdm(glob('out/sab_umls/*.nodes.csv')):
	with open(filename) as o:
		csv_reader = csv.reader(o)
		header = True
		for row in csv_reader:
			if header:
				header = False
			else:
				cid = row[0]
				name = cid if ":" in cid else row[1]
				uid = str(uuid5(NAMESPACE_URL, name))
				name_id_mapper[name] = uid
				id_mapper[cid] = uid

100%|██████████| 84/84 [00:43<00:00,  1.93it/s]


In [16]:
with open('out/name_id_mapper.json', "w") as o:
	o.write(json.dumps(name_id_mapper))

with open('out/id_mapper.json', "w") as o:
	o.write(json.dumps(id_mapper))

In [17]:
[i for i in id_mapper.keys() if ":" not in i][0]

'C3812978'

In [18]:
for filename in tqdm(glob('out/sab_umls/*.nodes.csv')):
	df = pd.read_csv(filename, index_col=0)
	new_index = []
	new_col = {}
	for i, row in df.iterrows():
		name = row["label"]
		if ":" in i:
			name = i
			col, val = i.replace(" CUI", "").split(":")
			if col not in new_col:
				new_col[col] = {"index": [], "values": []}
			new_col[col]["index"].append(i)
			new_col[col]["values"].append(str(val))
		new_index.append(name_id_mapper[name])
	for k,v in new_col.items():
		df.loc[v["index"], k] = v["values"]
	df.index = new_index
	df[~df.index.duplicated(keep='first')].to_csv(filename.replace("sab_umls", "sab"))
	

100%|██████████| 84/84 [04:37<00:00,  3.31s/it]


In [19]:
for filename in tqdm(glob('out/sab_umls/*.edges.csv')):
	new_filename = filename.replace("sab_umls", "sab")
	with open(filename) as o:
		csv_reader = csv.reader(o)
		with open(new_filename, 'w') as w:
			csv_writer = csv.writer(w)
			header = True
			for row in csv_reader:
				if header:
					header = False
					csv_writer.writerow(row)
				else:
					new_row = row
					new_row[1] = id_mapper[new_row[1]]
					new_row[2] = id_mapper[new_row[2]]
					csv_writer.writerow(new_row)


100%|██████████| 19/19 [03:32<00:00, 11.19s/it]


8196998 nodes
36046773 edges


In [20]:
len(set(id_mapper.values()))

8196986

In [21]:
list(id_mapper.keys())[0]

'GLYTOUCAN:G99760DU CUI'

In [22]:
id_mapper['GLYTOUCAN:G99760DU CUI']

'7c256cfc-61b7-5568-b997-eea76e864673'

In [2]:
all_nodes = set()
for filename in tqdm(glob('out/sab/*.nodes.csv')):
	with open(filename) as o:
		csv_reader = csv.reader(o)
		header = True
		for row in csv_reader:
			if header:
				header = False
			else:
				cid = row[0]
				all_nodes.add(cid)


100%|██████████| 84/84 [00:12<00:00,  6.53it/s]


In [3]:
len(all_nodes)

8196986

In [25]:
for filename in tqdm(glob('out/sab/*.edges.csv')):
	with open(filename) as o:
		csv_reader = csv.reader(o)
		header = True
		for row in csv_reader:
			if header:
				header = False
			else:
				if row[1] not in all_nodes:
					print(row)
					break
				if row[2] not in all_nodes:
					print(row)
					break

  0%|          | 0/19 [00:00<?, ?it/s]

100%|██████████| 19/19 [01:14<00:00,  3.93s/it]


In [10]:
rels = set()
with open('../dd_data/DD28Mar2024CSV/CUI-CUIs.csv') as o:
	csv_reader = csv.reader(o)
	for row in csv_reader:
		if 'C1537992' in row:
			rels.add(row[2])

In [13]:
rels

{'allelic_variant_of',
 'candidate_gene_tested_in',
 'chd',
 'chr_band_contains_gene',
 'coexpression_kidney___medulla',
 'component_of',
 'disease_causing_germline_mutation_s__in',
 'ds_3_prime',
 'expressed_in',
 'expresses',
 'gene_associated_with_disease_or_phenotype',
 'gene_disease_validity',
 'gene_product_of',
 'has_allelic_variant',
 'has_component',
 'has_gene_product',
 'has_human_ortholog',
 'has_marker_gene',
 'in_1_to_1_orthology_relationship_with',
 'inverse_candidate_gene_tested_in',
 'inverse_chr_band_contains_gene',
 'inverse_coexpression_kidney___medulla',
 'inverse_disease_causing_germline_mutation_s__in',
 'inverse_ds_3_prime',
 'inverse_gene_associated_with_disease_or_phenotype',
 'inverse_gene_disease_validity',
 'inverse_has_human_ortholog',
 'inverse_has_marker_gene',
 'inverse_in_1_to_1_orthology_relationship_with',
 'inverse_is_directional_form_of',
 'inverse_is_feature_type',
 'inverse_is_gene_biotype',
 'inverse_isa',
 'inverse_overlaps',
 'inverse_pathway_